In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("kidsInMindSubtitles2004.csv")

In [3]:
df[:5].to_csv("sample_kidsinmind.csv", index = False)

In [4]:
for col in df.columns:
    print(col)

Unnamed: 0.1
Unnamed: 0
Movie_Title
Sex_Nudity
Violence_Gore
Language
Movie_Description
Sex_Nudity_Description
Violence_Gore_Description
Language_Description
Substance_Use_Description
Discussion_Topics_Description
Message
HyperLink
subtitles


In [230]:
#!pip install ohmeow-blurr -q
#!pip install bert-score -q

In [231]:
from fastai.text.all import *
from blurr.text.data.all import *

In [232]:
df = df.dropna().reset_index()

In [233]:
df

,index,Unnamed: 0.1,Unnamed: 0,Movie_Title,Sex_Nudity,Violence_Gore,Language,Movie_Description,Sex_Nudity_Description,Violence_Gore_Description,Language_Description,Substance_Use_Description,Discussion_Topics_Description,Message,HyperLink,subtitles
0,0,1,1,Abduction,4,5,5,"After a teenager (Taylor Lautner) discovers that who he thought were his parents are actually CIA agents assigned to keep him safe, his world quickly comes unraveled. He then sets out to find the truth about his life. Also with Lily Collins, Alfred Molina, Maria Bello and Sigourney Weaver. Directed by John Singleton. [1:46]","A teen boy and a teen girl share a lingering, passionate kiss; the girl wraps her legs around the boy's waist and he lifts her up, he sits down, the girl straddles him and they continue kissing until the girl pulls away.\n► Teen girls and boys are seen in a swimming pool; the girls wear cleavage-exposing bikinis and the bare chests and backs of the boys are visible. The bare chest and back of a teen boy is visible as he awakens shirtless, and we see him walking around shirtless. On two occasions we see the bare abdomen of a teen girl as she wears her shirt tied up. A teen girl wears a low-...","Two snipers shoot and kill five men (no blood is visible); we see the bodies of the men hit the ground and later see a man standing over the body of one of them (no blood is visible). A SWAT team storms into an apartment where six men are seen sitting or sleeping, and the team opens fire, presumably killing all of the apartment's occupants.\n► Two gunmen fire machine guns into a restaurant where a man, a woman and a teen boy and teen girl sit; the gunmen enter the room and see the man and the woman slumped over, appearing to be dead (they are unharmed), the man and the woman grab guns and ...","At least 1 F-word, 4 scatological terms (1 mild), 4 anatomical terms, 3 mild obscenities, name-calling (freak, psycho, foolish, boring, stubborn, little man), 4 religious exclamations.","Teenage boys and girls are seen drinking unidentified alcoholic drinks at a party, a teen boy drinks to the point of drunkenness and we see him passed out with a cup in his hand, a teen boy tells his father he was ""hangover,"" a teen girl teases a teen boy about being so intoxicated at a party that he passed out in his underwear, a woman warns her teen son that she does not want him to get in a drunk driving accident, a bottle of wine with two glasses is seen in front of a man and a woman, we hear lyrics of a song that say ""I want to drink your wine,"" a teen boy sells other teens fake IDs (...","The CIA, disappearances, child abductions, trust, dishonesty, jealousy, disappointment, finding one's family, adoption, espionage, revenge.",It can be hard to trust people when you are unsure of their background.,https://kids-in-mind.com/a/abduction.htm,"Advertise your product or brand here\ncontact www.OpenSubtitles.org todayNATHAN: Let's go party![WHOOPING]Yeah, baby!Yeah! Gilly,\nget out here!BOTH: Yeah! Yeah!Let's go, baby,\nwe got bitches waiting![BOTH EXCLAIMING]Go! Faster!\nPut the pedal\nto the metal!Party!BOTH: Go! Go! Go!Yeah, Jake! [WHOOPS]Yeah!NATHAN: Hey, ladies.\nWe are here.You guys are crazy!Get ready to party.Yeah!We are here!GILLY: Whoa! [LAUGHING]Yo, Nathan.Dude, you're crazy.Yes, I am.You all right, man? Okay.Yeah, I'm good.Let's do this.Let's go.[CROWD CLAMORING]Oh, check it out!Dude.BOY: Gilly!Yo, guys,\nbe right back..."
1,1,3,3,About Last Night,7,2,10,"Two couples (Keven Hart and Regina Hall, and Michael Ealy and Joy Bryant) meet in a bar. They explore new love and casual sex, but are stymied by the realities of daily life. Also with Paula Patton and Christopher McDonald. Directed by Steve Pink. [1:40]",A bedroom scene features a man sitting upright (we see his bare chest) thrusting and a woman under covers with only shoulders and arms revealed; she is grunting and gasping and becomes angry because she has a Charlie-horse and the scene ends.\n► A man and 

In [234]:
#!pip install profanity-check

In [236]:
df['Language_Description'][0]

'At least 1 F-word, 4 scatological terms (1 mild), 4 anatomical terms, 3 mild obscenities, name-calling (freak, psycho, foolish, boring, stubborn, little man), 4 religious exclamations.'

In [238]:
df['subtitles']=df['subtitles'].astype(str)

In [240]:
df['subtitles'] = df['subtitles'].apply(lambda x: x.replace('/',''))
df['subtitles'] = df['subtitles'].apply(lambda x: x.replace('\xa0',''))

In [241]:
#!pip install ohmeow-blurr -Uqq

In [242]:
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [243]:
#!pip install transformers -Uqq

In [244]:
from transformers import BartForConditionalGeneration

In [245]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name,
                                                                  model_cls=BartForConditionalGeneration)

In [246]:
text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization');

In [247]:
text_gen_kwargs={'max_length': 248,
 'min_length': 56,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False}

In [248]:
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(
    hf_arch, hf_config, hf_tokenizer, hf_model, max_length=256, max_tgt_length=130, text_gen_kwargs=text_gen_kwargs
)

blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('subtitles'), get_y=ColReader('Language_Description'), splitter=RandomSplitter())

In [249]:
dls = dblock.dataloaders(df, bs=2)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [250]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [252]:
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls,
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt()
learn.freeze()

/usr/local/lib/python3.8/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the

In [254]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
learn.fit_one_cycle(15, lr_max=2.2908675418875646e-06, cbs=fit_cbs)

In [34]:
learn.save("BartForSummarization_v1")

In [163]:
loaded_dict = torch.load('models/BartForSummarization.pth')

In [164]:
model_state_dict = loaded_dict['model']

In [165]:
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [166]:
model.eval()

BaseModelWrapper(
  (hf_model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50264, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50264, 1024, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0): BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_feat

In [73]:
#df['subtitles'][0]

In [174]:
import pandas as pd

In [201]:
df_predictions = pd.read_csv("test_dataset__bart_model_language_rating_prediction.csv")

In [202]:
df_predictions.head(2)

,Language_Score,Predcited_Score,subtitle,language_summary,cleaned_text,inappropriate_words,inappropriate_words_count
0,6,6.543292,"The High Council\nhas reached a verdict --guilty!You taught the K. O.K.\nhandshake to a girl.Dude, what were\nyou thinking?Boo-la.\nBoo-la. Boo-la.Boo-la. Boo-la.Boo-la. Boo-la.Boo-la. Boo-la. Boo-la.\nBoo-la. Boo-la.Spence, are you done yet?\nWe need this......room for the bobsled.Jiminy Cricket.Goddamn it, Dave!Mush, I say!-- Mush!\n-- Ow!Oh, come on!Change of plans, boys.\nSomeone ralphed all over\nthe living room.We need to relocate.\nYou guys, out.Hey, I will not tolerate\ninsubordinationduring my ceremony.Oh, just relax.Look, I thought that your mask\nwas really scary this time.You t...","13 F-words, 1 nearly uttered F-word, 3 obscene hand gestures, 9 sexual references, 12 scatological terms, 29 anatomical terms (the name of a fraternity house is KOK so whenever it is uttered it sounds like an anatomical term), 3 mild obscenities, 3 derogatory terms for lesbians, 2 religious profanities, 10 religious exclamations, and many insults.\n","the high council reached verdict --guilty!you taught k. o.k. handshake girl.dude, thinking?boo-la. boo-la. boo-la.boo-la. boo-la.boo-la. boo-la.boo-la. boo-la. boo-la. boo-la. boo-la.spence, yet? we need this......room bobsled.jiminy cricket.goddamn it, dave!mush, i say!-- mush! -- ow!oh, come on!change plans, boys. someone ralphed living room.we need relocate. you guys, out.hey, i tolerate insubordinationduring ceremony.oh, relax.look, i thought mask scary time.you ""fraternity brothers"" think wantin house, right? wrong.i'm president!dave?what's that, adam?i think it's time president got l...","['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'goddamn', 'fat', 'shit', 'masturbate', 'hell', 'fuck', 'fat', 'fat', 'toilet', 'dead', 'skank', 'prick', 'ass', 'fuck', 'cock', 'toilet', 'toilet', 'toilet', 'fucking', 'tinkle', 'ugly', 'butt', 'stupid', 'stupid', 'hell', 'fucked', 'killing', 'fuck', 'sex', 'tits', 'bulldyke', 'damn', 'fat', 'stupid', 'ass', 'fucking', 'piss', 'desire', 'desire', 'desire', 'burn', 'tongue']",47
1,7,7.480484,"Danny, give me my shirt!Come and get it.Don't you think that maybe we're taking\nthis hazing thing a little bit too far?I absolutely do not.Hey!Cassidy!- Hey, give me a beer. Come on, man.\n- Get your own, pretty boy.- Don't be a dick.\n- Get your own.Hey, move it, douche bag. Get out of here.Mmm. Sudsy!Hey, Chugs! Give me a beer! And a BJ.Hey, what the fuck?Hey, no! You both, out! Now!Some massive titties!How long have you been a Theta?- Do you want one?\n- No, thanks.- Hey, Ellie, up here.\n- I got to go. Sorry.It wasn't that hard a question.Asshole!- Where are we going, Claire?\n- Hurry...","About 25 F-words, 2 obscene hand gestures, 8 sexual references, 10 scatological terms, 13 anatomical terms, 23 mild obscenities, name-calling (dummy, psycho, freak show, retarded, dumb, jerk, idiot, ""are you deaf or stupid,"" ugly), 17 religious exclamations.\n","danny, shirt!come it.don't think maybe we're taking hazing thing little bit far?i absolutely not.hey!cassidy!- hey, beer. come on, man. - get own, pretty boy.- don't dick. - get own.hey, it, douche bag. get here.mmm. sudsy!hey, chugs! give beer! and bj.hey, fuck?hey, no! you both, out! now!some massive titties!how long theta?- do want one? - no, thanks.- hey, ellie, here. - i got go. sorry.it wasn't hard question.asshole!- where going, claire? - hurry up. jessica's waiting.goodbye, virginity! yeah! theta pi rules!i think i'm gonna sick. look below!heather, room.hey, boys. out, out, out!- i...","['douche', 'damn', 'bitch', 'bra', 'sex', 'laid', 'shit', 'fuck', 'puke', 'bitch', 'fucking', 'died', 'fucking', 'fucking', 'died', 'hell', 'fuck', 'stupid', 'destroy', 'attack', 'destroy', 'kill', 'killer', 'fuck', 'dead', 'fucking', 'hell', 'kill', 'hole', 'killed', 'god', 'stupid', 'devil', 'ass', 'kill', 'fuck', 'kill', 'hell', 'screw', 'rape', 'burn', 'kill', 'hell', 'killed', 'dead', 'dead'

In [203]:
df_predictions['bart'] = None

In [204]:
from tqdm import tqdm

In [205]:

bart_predictions_list = []

for idx in tqdm(range(len(df_predictions))):
    
    
    query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][idx], 1))+" \
    ; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][idx] + " Generate \
    language summary like kids in mind website? Please consider the below two example language summaries of \
    Please consider the below two example language summaries of movies \
    movies:  \n 1." + df['Language_Description'][0] + "\n 2."+df['Language_Description'][1]

    query = query.replace("[", "")
    query = query.replace("]", "\n\n")
    
    # print("-----------")
    # print(query)
    
    outputs = learn.blurr_generate(query, early_stopping=False, num_return_sequences=1)
    
    #df_predictions['bart'][idx] = outputs[0]['generated_texts']
    
    bart_predictions_list.append(outputs[0]['generated_texts'])
    
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201/201 [04:55<00:00,  1.47s/it]


In [206]:
#bart_predictions_list

In [6]:
df_predictions.head(2)

,Oringinal Summary,Predicted Summary
0,"13 F-words, 1 nearly uttered F-word, 3 obscene...","About 67 F-words and its derivatives, 2 obsce..."
1,"About 25 F-words, 2 obscene hand gestures, 8 s...","About 67 F-words and its derivatives, 2 obsce..."


In [7]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [8]:

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in range(len(df_predictions)):
    
    scores = scorer.score(df_predictions['Oringinal Summary'][idx], df_predictions['Predicted Summary'][idx])
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [9]:
sum(rougeL_scores)/len(rougeL_scores)

0.18237093437835214

In [11]:
import bert_score

In [12]:
references = list(df_predictions['Oringinal Summary'])
summaries = list(df_predictions['Predicted Summary'])

In [14]:
P, R, F1 = bert_score.score(summaries, references, lang='en')
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.8223400115966797
BERTScore Recall: 0.8944635391235352
BERTScore F1: 0.8561439514160156
